<a href="https://colab.research.google.com/github/Jay0073/Transformer-Based-NLP/blob/main/NER-with-PretrainedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition with pretrained model


In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00


In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments, BertForTokenClassification, BertTokenizer
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from datasets import ClassLabel, Sequence

In [3]:
# Load text and labels from separate files
def load_data(text_file, labels_file):
    with open(text_file, 'r') as f:
        sentences = f.readlines()

    with open(labels_file, 'r') as f:
        label_lines = f.readlines()

    # Create aligned lists
    sentences = [s.strip().split() for s in sentences]  # Split words in each sentence
    labels = [l.strip().split() for l in label_lines]  # Split labels for each sentence

    return sentences, labels

In [4]:
# Load your data
sentences, labels = load_data('/content/text_train.txt', '/content/labels_train.txt')

In [5]:
sentences[1]

['Despite',
 'Bangladesh',
 "'s",
 'highest',
 'total',
 'ever',
 'in',
 'a',
 'limited-overs',
 'match',
 ',',
 'the',
 'Kiwis',
 'were',
 'able',
 'to',
 'win',
 'the',
 'match',
 'by',
 'six',
 'wickets',
 'in',
 'Auckland',
 '.']

In [6]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

In [7]:
tokenizer

BertTokenizer(name_or_path='dbmdz/bert-large-cased-finetuned-conll03-english', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [8]:
# Tokenize and align labels
def tokenize_and_align_labels(sentences, labels):
    tokenized_inputs = []
    aligned_labels = []
    for sentence, label in zip(sentences, labels):
        encodings = tokenizer(sentence, truncation=True, padding='max_length', is_split_into_words=True, return_tensors='pt')
        # Align labels with tokenized input
        label_ids = [-100] * len(encodings["input_ids"][0])  # Default labels to -100
        for j in range(len(label)):
            if j < len(encodings["input_ids"][0]) - 2:  # Adjust for [CLS] and [SEP]
                label_ids[j + 1] = label[j] if label[j] != "O" else -100  # Retain 'O' labels as -100
        tokenized_inputs.append(encodings)
        aligned_labels.append(label_ids)

    return tokenized_inputs, aligned_labels

In [9]:
inputs, labels = tokenize_and_align_labels(sentences, labels)

In [10]:
# Tokenized input
inputs[0]

{'input_ids': tensor([[  101,  1203,  2512,   112,   188,  5428,  1264,  1144,  2297,   170,
         22407,   118, 14112,  1158,  1782,  1166,  6735,  1107,  1103,  1148,
          1104,  1210,  1141,   118,  1285,  1835,  1116,  1107,  1203,  2512,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [11]:
# Split the data into training and testing sets (80% training, 20% testing)
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    inputs,
    labels,
    test_size=0.2,  # 20% for testing
    random_state=12  # For reproducibility
)

In [12]:
# Load the BERT model for token classification
model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
class NERDataset(Dataset):
    def __init__(self, encodings, labels, num_labels): # Add num_labels as an argument
        self.encodings = encodings
        self.labels = labels
        self.num_labels = num_labels  # Store num_labels

        # Get unique labels and map them to IDs, limiting to num_labels
        unique_labels = set([label for sublist in labels for label in sublist])
        unique_labels = list(unique_labels)[:num_labels] # Limit unique labels to num_labels
        self.label2id = {label: i for i, label in enumerate(unique_labels)}
        self.id2label = {i: label for label, i in self.label2id.items()}

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings[idx]['input_ids'][0],
            'attention_mask': self.encodings[idx]['attention_mask'][0],
        }
        # Convert labels to numerical IDs using the label2id mapping
        item['labels'] = torch.tensor([self.label2id.get(label, -100) for label in self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
# Get the number of labels from the pre-trained model
num_labels = model.num_labels

In [15]:
# Create dataset objects
train_dataset = NERDataset(train_inputs, train_labels, num_labels)
test_dataset = NERDataset(test_inputs, test_labels, num_labels)

In [16]:
train_dataset

In [17]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: beyondtechwithrj (beyondtechwithrj-siddartha-institute-of-technology-and-s) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.848100,0.834477
2,0.641600,0.803900


Epoch,Training Loss,Validation Loss
1,0.848100,0.834477
2,0.641600,0.803900
3,0.409800,0.869561


TrainOutput(global_step=4959, training_loss=0.6989407594176544, metrics={'train_runtime': 7355.7494, 'train_samples_per_second': 2.697, 'train_steps_per_second': 0.674, 'total_flos': 1.842226911302861e+16, 'train_loss': 0.6989407594176544, 'epoch': 3.0})

In [20]:
trainer.evaluate()

{'eval_loss': 0.8695614337921143,
 'eval_runtime': 166.1504,
 'eval_samples_per_second': 9.949,
 'eval_steps_per_second': 2.492,
 'epoch': 3.0}

In [21]:
model.save_pretrained('./my_finetuned_ner_model')
tokenizer.save_pretrained('./my_finetuned_ner_model')

('./my_finetuned_ner_model/tokenizer_config.json',
 './my_finetuned_ner_model/special_tokens_map.json',
 './my_finetuned_ner_model/vocab.txt',
 './my_finetuned_ner_model/added_tokens.json')

In [22]:
# Load the fine-tuned model and tokenizer
model_path = './my_finetuned_ner_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

In [23]:
# Create a pipeline for named entity recognition
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [24]:
# Example input text
text = "My name is Wolfgang and I live in Berlin"

In [25]:
# Perform NER
ner_results = nlp(text)

# Print the results
ner_results

[{'entity': 'B-ORG',
  'score': 0.9280634,
  'index': 1,
  'word': 'My',
  'start': 0,
  'end': 2},
 {'entity': 'B-ORG',
  'score': 0.96664286,
  'index': 2,
  'word': 'name',
  'start': 3,
  'end': 7},
 {'entity': 'B-ORG',
  'score': 0.88486457,
  'index': 3,
  'word': 'is',
  'start': 8,
  'end': 10},
 {'entity': 'B-ORG',
  'score': 0.9903867,
  'index': 4,
  'word': 'Wolfgang',
  'start': 11,
  'end': 19},
 {'entity': 'B-MISC',
  'score': 0.9841977,
  'index': 5,
  'word': 'and',
  'start': 20,
  'end': 23},
 {'entity': 'B-MISC',
  'score': 0.9961606,
  'index': 6,
  'word': 'I',
  'start': 24,
  'end': 25},
 {'entity': 'B-MISC',
  'score': 0.998005,
  'index': 7,
  'word': 'live',
  'start': 26,
  'end': 30},
 {'entity': 'B-MISC',
  'score': 0.99867886,
  'index': 8,
  'word': 'in',
  'start': 31,
  'end': 33},
 {'entity': 'B-MISC',
  'score': 0.9991887,
  'index': 9,
  'word': 'Berlin',
  'start': 34,
  'end': 40}]

In [ ]:
ner_results[].keys()